In [1]:
import os
import numpy as np
import pandas as pd
import glob
from prediction_utils.util import yaml_read, df_dict_concat

In [2]:
table_path = '../figures/hyperparameters/'
os.makedirs(table_path, exist_ok = True)

In [3]:
param_grid_base = {
        "lr": [1e-3, 1e-4, 1e-5],
        "batch_size": [128, 256, 512],
        "drop_prob": [0.0, 0.25, 0.5, 0.75],
        "num_hidden": [1, 2, 3],
        "hidden_dim": [128, 256],
}

the_dict = {'hyperparameter': [], 'Grid': []}
for key, value in param_grid_base.items():
    the_dict['hyperparameter'].append(key)
    the_dict['Grid'].append(value)
the_df = pd.DataFrame(the_dict)
rename_grid = {
    'hyperparameter': ['lr', 'batch_size', 'drop_prob', 'num_hidden', 'hidden_dim'],
    'Hyperparameter': ['Learning Rate', 'Batch Size', 'Dropout Probability', 'Number of Hidden Layers', 'Hidden Dimension']
}
rename_df = pd.DataFrame(rename_grid)
the_df = the_df.merge(rename_df)[['Hyperparameter', 'Grid']].sort_values('Hyperparameter')

In [4]:
the_df

,Hyperparameter,Grid
1,Batch Size,"[128, 256, 512]"
2,Dropout Probability,"[0.0, 0.25, 0.5, 0.75]"
4,Hidden Dimension,"[128, 256]"
0,Learning Rate,"[0.001, 0.0001, 1e-05]"
3,Number of Hidden Layers,"[1, 2, 3]"


In [5]:
the_df.to_latex(os.path.join(table_path, 'param_grid.txt'), index=False)

In [6]:
selected_models_path = '/share/pi/nigam/projects/spfohl/cohorts/admissions/optum/experiments/baseline_tuning_fold_1/config/selected_models'

In [7]:
selected_models_path_dict = {
    'starr': '/share/pi/nigam/projects/spfohl/cohorts/admissions/starr_20200523/experiments/baseline_tuning_fold_1_10/config/selected_models',
    'mimic': '/share/pi/nigam/projects/spfohl/cohorts/admissions/mimic_omop/experiments/baseline_tuning_fold_1_10/config/selected_models',
    'optum': '/share/pi/nigam/projects/spfohl/cohorts/admissions/optum/experiments/baseline_tuning_fold_1/config/selected_models',
}
selected_param_dict = {
    db: {
        task: yaml_read(glob.glob(os.path.join(db_path, task, '*.yaml'), recursive=True)[0]) for task in os.listdir(db_path)
    }
    for db, db_path in selected_models_path_dict.items()
}

In [8]:
col_order = {
    'starr': ['hospital_mortality', 'LOS_7', 'readmission_30'],
    'mimic': ['los_icu_3days', 'los_icu_7days', 'mortality_hospital', 'mortality_icu'],
    'optum': ['readmission_30', 'LOS_7'],
}

In [9]:
for db in selected_param_dict.keys():
    db_params = selected_param_dict[db]
    db_df = (
        pd.concat({key: pd.DataFrame(value, index= [0]) for key, value in db_params.items()})
        .reset_index(level=1,drop=True)
        .rename_axis('task')
        .transpose()
        .rename_axis('hyperparameter')
        .reset_index()
        .merge(rename_df, how ='right')
    )
    db_df = db_df[['Hyperparameter'] + list(set(db_df.columns) - set(['hyperparameter', 'Hyperparameter']))].sort_values('Hyperparameter')
    db_df = db_df[['Hyperparameter'] + col_order[db]].sort_values('Hyperparameter')
    selected_param_path = os.path.join(table_path, db)
    os.makedirs(selected_param_path, exist_ok=True)
    db_df.to_latex(os.path.join(selected_param_path, 'selected_param_table.txt'), index=False)